In [1]:
import pandas as pd
import numpy as np

In [2]:
def validate_isin(column,values_to_check,df):
    isin_validation = df[column].isin(values_to_check)
    return isin_validation

def validate_pattern(column,pattern,df):
    pattern_matched = df[column].str.match(pattern)
    return pattern_matched
    
def validate_bounded(column, lower_bound, upper_bound, df):
    bounded_range = (df[column] >= lower_bound) & (df[column] < upper_bound)
    return bounded_range

    

In [3]:
df1= pd.read_csv('GooglePlay.csv')
duplicates = df1[df1.duplicated(subset=['App'], keep=False)]
print("Duplicated contents in 'App':")
print(duplicates)


Duplicated contents in 'App':
       Unnamed: 0                                             App   
1               1                             Coloring book moana  \
23             23                          Mcqueen Coloring pages   
36             36  UNICORN - Color By Number & Pixel Art Coloring   
42             42                      Textgram - write on photos   
139           139                            Wattpad 📖 Free Books   
...           ...                                             ...   
10714       10715                              FarmersOnly Dating   
10719       10720              Firefox Focus: The privacy browser   
10729       10730                                     FP Notebook   
10752       10753                  Slickdeals: Coupons & Shopping   
10767       10768                                            AAFP   

                  Category  Rating  Reviews                Size      Installs   
1           ART_AND_DESIGN     3.9      967                 

In [5]:
df1= pd.read_csv('GooglePlay.csv')

invalid={}
valid_Type= ['Free', 'Paid']
df=df1[df1['Type' ].notna()]
invalid["Type"] = df[~validate_isin('Type',valid_Type, df)]

valid_Category=['FAMILY', 'GAME', 'TOOLS', 'MEDICAL', 'BUSINESS', 'PRODUCTIVITY', 'PERSONALIZATION', 'COMMUNICATION','SPORTS', 'LIFESTYLE', 'FINANCE', 'HEALTH_AND_FITNESS', 'PHOTOGRAPHY','SOCIAL', 'NEWS_AND_MAGAZINES','SHOPPING', 'TRAVEL_AND_LOCAL','DATING', 'BOOKS_AND_REFERENCE', 'VIDEO_PLAYERS', 'EDUCATION','ENTERTAINMENT', 'MAPS_AND_NAVIGATION', 'FOOD_AND_DRINK', 'HOUSE_AND_HOME','AUTO_AND_VEHICLES','LIBRARIES_AND_DEMO', 'WEATHER', 'ART_AND_DESIGN', 'EVENTS', 'PARENTING','COMICS','BEAUTY']
df=df1[df1['Category' ].notna()]
invalid["Category"]= df[~validate_isin('Category',valid_Category, df)]

df=df1[df1['Installs' ].notna()]
valid_Installs = '\d{1,3}(,\d{3})*\+|0'
invalid['Installs'] = df1[~validate_pattern('Installs',valid_Installs, df)]

df=df1[df1['Last Updated' ].notna()]
pattern_Last_Updated= '\d{1,2}-[(Jan)|(Aug)|Dec|Feb|Jul|Apr|Aug|Mar|May|Nov|Sep|Oct]{3}-\d{2}'
invalid['Last Updated'] = df[~validate_pattern('Last Updated',pattern_Last_Updated, df)]

df=df1[df1['Price' ].notna()]
pattern_Price= '(\$?\d+(\.\d{2})?|0)'
invalid['Price']= df[~validate_pattern('Price',pattern_Price, df)]

df=df1[df1['Genres' ].notna()]
pattern_Genres= '([A-Z][a-z]* ?(&;)?)+' 
invalid['Genres'] = df[~validate_pattern('Genres',pattern_Genres, df)]

valid_Content_Rating= ['Everyone','Teen', 'Mature 17+', 'Everyone 10+', 'Adults only 18+', 'Unrated']
df=df1[df1['Content Rating' ].notna()]
invalid['Content Rating'] = df[~validate_isin('Content Rating',valid_Content_Rating, df)]

df=df1[df1['Rating' ].notna()]
df=df[df['Rating'].isnull()]
invalid['Rating'] = df[~validate_bounded('Rating',0,5, df)]

df=df1[df1['Reviews' ].notna()]
invalid['Reviews'] = df[~validate_bounded('Reviews',0,np.inf, df)]

df=df1[df1['Size' ].notna()]
pattern_Size='\d+(\.\d+)?[MK]?'
invalid['Size'] = df[~validate_pattern('Size',pattern_Size, df)]

df1['Current Ver'] = df1['Current Ver'].astype(str)

df=df1[df1['Current Ver' ].notna()]
valid_Current_Ver = '^\d+(\.\d+)*'
invalid['Current Ver'] = df[~validate_pattern('Current Ver',valid_Current_Ver, df)]

df1['Android Ver'] = df1['Android Ver'].astype(str)

df=df1[df1['App' ].notna()]

df=df1[df1['Android Ver' ].notna()]
valid_Android_Ver = '\d+(\.\d+)*\s*-\s*\d+(\.\d+)*|\d+(\.\d+)*\s+and\s+up'
invalid['Android Ver' ]= df[~validate_pattern('Android Ver',valid_Android_Ver, df)]




In [9]:
quality_df = pd.DataFrame(index=list(df1.columns),columns=['Consistency', 'Currentness', 'Validity', 'Completeness', 'Accuracy', 'Null Values', 'Records'])

#Functions to check inconsistencies
def validate_ver_inconsistencies(df1):
    inconsistencies = []
    for index, row in df1.iterrows():
        current_ver = str(row["Current Ver"])  
        android_ver = str(row["Android Ver"])  
        
        if "Varies with device" in android_ver:
            continue  
        
        android_ver_range = android_ver.split(" and up")[0]
        android_ver_min = android_ver_range.split(" - ")[0]
        
        if android_ver_min and current_ver:
            if android_ver_min not in current_ver:
                inconsistencies.append((current_ver, android_ver))
        elif not current_ver:
            inconsistencies.append(("Missing", android_ver))    
    return inconsistencies



###################################

def validate_type_consistency(row):
    if row['Type'] == 'free' and row['Price'] != 0:
        return False
    elif row['Type'] == 'paid' and row['Price'] <= 0:
        return False
    else:
        return True
####################################  
def check_install_review_consistency(row, threshold=0.5):
    installs = row['Installs']
    reviews = row['Reviews']
    
    installs = int(installs.replace('+', '').replace(',', ''))
    reviews = int(reviews)
    
    if installs == 0:
        return False
    
    review_ratio = reviews / installs

    if review_ratio <= threshold:
        return True
    else:
        return False

count = 0
for index, row in df1.iterrows():
    is_consistent = check_install_review_consistency(row)
    if not is_consistent:
        print(f"Row {index}: Consistent = {is_consistent}, Installs = {row['Installs']}, Reviews = {row['Reviews']}")
        count += 1

print(count)
     
#############################################
def validate_genre_consistency(row):
    category = row['Category']
    genres = row['Genres']
    if category.lower().replace('_', ' ').replace('and', '&') in genres.lower():
        return True
    else:
        return False

genre_consistency_mask = df1.apply(validate_genre_consistency, axis=1)

inconsistent_rows = df1[~genre_consistency_mask]

print("Rows where Genres and Category are inconsistent:")
print(inconsistent_rows)

percentage_genre_consistency = (genre_consistency_mask.sum() / len(df1)) * 100

  



Row 2450: Consistent = False, Installs = 5+, Reviews = 4
Row 2454: Consistent = False, Installs = 1+, Reviews = 4
Row 4465: Consistent = False, Installs = 0+, Reviews = 0
Row 4550: Consistent = False, Installs = 1+, Reviews = 4
Row 4556: Consistent = False, Installs = 100,000+, Reviews = 51068
Row 4663: Consistent = False, Installs = 10,000+, Reviews = 10249
Row 4702: Consistent = False, Installs = 1,000+, Reviews = 594
Row 4946: Consistent = False, Installs = 10,000+, Reviews = 5639
Row 5245: Consistent = False, Installs = 10+, Reviews = 9
Row 5307: Consistent = False, Installs = 0+, Reviews = 0
Row 5486: Consistent = False, Installs = 0+, Reviews = 0
Row 5500: Consistent = False, Installs = 1+, Reviews = 1
Row 5812: Consistent = False, Installs = 1+, Reviews = 2
Row 5854: Consistent = False, Installs = 10+, Reviews = 8
Row 5917: Consistent = False, Installs = 1+, Reviews = 2
Row 5945: Consistent = False, Installs = 0+, Reviews = 0
Row 6046: Consistent = False, Installs = 10+, Reviews

In [11]:
df1 = df1.drop(columns=df1.filter(like='Unnamed').columns)
sum=0
quality_df = pd.DataFrame(index=list(df1.columns),columns=['Consistency', 'Currentness', 'Validity', 'Completeness', 'Accuracy', 'Null Values', 'Records'])

for column in df1.columns:
    quality_df['Records'][column] = len(df1)

for column in df1.columns: 
        quality_df['Null Values'][column]=df1[column].isnull().sum()

for column in df1.columns:
    missing = df1[column].isnull().sum()
    total_valid = len(df1) - missing
    if total_valid == 0:
        accuracy = 0
    elif column in invalid:
        accuracy = ((total_valid - len(invalid[column])) / len(df1)) * 100
    else:
        accuracy = 100
    quality_df.loc[column, 'Accuracy'] = accuracy

        
for column in df1.columns:
        missing = df1[column].isnull().sum()
        sum = (((len(df1[column]) - missing) / len(df1[column])) * 100)
        quality_df['Completeness'][column] = sum

for column in df1.columns:
    missing = df1[column].isnull().sum()
    if column in invalid:
        quality_df['Validity'][column]  = (1- (len(invalid[column])) / len(df1[column])) * 100
    else:
        quality_df['Validity'][column] = 100


for column in df1.columns:
    missing = df1[column].isnull().sum()
    total_valid = len(df1) - missing
    if total_valid == 0:
        accuracy = 0
    elif column in invalid:
        accuracy = ((total_valid - len(invalid[column])) / len(df1)) * 100
    else:
        accuracy = 100
    quality_df.loc[column, 'Accuracy'] = accuracy

valid_dates = df[validate_pattern('Last Updated',pattern_Last_Updated, df)]
valid_dates['Year'] = valid_dates['Last Updated'].apply(lambda x: int(x[-2:]))
valid_dates_before_2018 = valid_dates[valid_dates['Year'] < 17]
percentage_valid_before_2017 = (len(valid_dates_before_2018) / len(valid_dates)) * 100
for column in df1.columns:
    if column != 'Unnamed: 0':  
        quality_df['Currentness'][column]=percentage_valid_before_2017

mask = df1.apply(validate_type_consistency, axis=1)      
quality_df['Consistency']['Type']= (1-((len(df) - mask.sum()) / len(df) ))* 100
quality_df['Consistency']['Price']= (1-((len(df) - mask.sum()) / len(df) ))* 100
mask = df1.apply(check_install_review_consistency, axis=1)      
quality_df['Consistency']['Installs']= (1-((len(df) - mask.sum()) / len(df) ))* 100
quality_df['Consistency']['Reviews']= (1-((len(df) - mask.sum()) / len(df) ))* 100
mask = df1.apply(validate_genre_consistency, axis=1)      
quality_df['Consistency']['Category']= (1-((len(df) - mask.sum()) / len(df) ))* 100
quality_df['Consistency']['Genres']= (1-((len(df) - mask.sum()) / len(df) ))* 100
# quality_df['Consistency']['App']= 100
# quality_df['Consistency']['Rating']= 100
# quality_df['Consistency']['Last Updated']=100
# quality_df['Consistency']['Content Rating']=100
quality_df = quality_df.fillna(100)
quality_df

,Consistency,Currentness,Validity,Completeness,Accuracy,Null Values,Records
App,100.000000,14.98155,100.000000,100.000000,100.000000,0,10840
Category,71.263838,14.98155,100.000000,100.000000,100.000000,0,10840
Rating,100.000000,14.98155,100.000000,86.402214,86.402214,1474,10840
Reviews,99.474170,14.98155,100.000000,100.000000,100.000000,0,10840
Size,100.000000,14.98155,84.363469,100.000000,84.363469,0,10840
Installs,99.474170,14.98155,100.000000,100.000000,100.000000,0,10840
Type,100.000000,14.98155,100.000000,99.990775,99.990775,1,10840
Price,100.000000,14.98155,100.000000,100.000000,100.000000,0,10840
Content Rating,100.000000,14.98155,100.000000,100.000000,100.000000,0,10840
Genres,71.263838,14.98155,100.000000,100.000000,100.000000,0,10840


In [12]:
df2 = pd.read_csv("Playstore_final.csv")

C:\Users\BlueWindows\AppData\Local\Temp\ipykernel_3196\676857778.py:1: DtypeWarning: Columns (7,12,29,30,33,35,37,38,40,42,43,44,46,47,48,49,50,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("Playstore_final.csv")


In [13]:
# list(df2['Version'].value_counts().keys())

dataset-2

In [14]:
invalids={}
valid_Free= [True, False]
df_2=df2[df2['Free' ].notna()]
invalids["Free"] = df_2[~validate_isin('Free',valid_Free, df_2)]

df_2=df2[df2['Version' ].notna()]
valid_Version = '[A-Za-z.-_\s]*?\s*?_*?-*?.*?v?\d+?(\.\d+)*?'#'[A-Za-z]*?\s?-?\s?(\d+(\.\d+)*)?([-_][a-zA-Z0-9]+)*'  #'[A-Za-z0-9_]*-*v?(\d+(\.\d+)*)?'  
invalids['Version' ]= df_2[~validate_pattern('Version',valid_Version, df_2)]

df_2=df2[df2['Developer Internal ID' ].notna()]
df_2['Developer Internal ID'] = df_2['Developer Internal ID'].astype(str)
valid_Developer_Internal_ID = '[0-9]+(?:\.[0-9]+)?(?:[eE][+-]?[0-9]+)?'
invalids['Developer Internal ID' ]= df_2[~validate_pattern('Developer Internal ID',valid_Developer_Internal_ID, df_2)]


df_2=df2[df2['Android version Text' ].notna()] 
valid_Android_version_Text = '\d+(\.\d+)*(\s*-\s*\d+(\.\d+)*)?W*(\d+(\.\d+)*)?\s*(and\s*up)?'#\d+(\.\d+)*\s*-\s*\d+(\.\d+)*|\d+(\.\d+)*\s+and\s+up'
invalids['Android version Text' ]= df_2[~validate_pattern('Android version Text',valid_Android_version_Text, df_2)]

valid_Editor_Choice= [True, False]
df_2=df2[df2['Editor Choice' ].notna()]
invalids["Editor Choice"] = df_2[~validate_isin('Editor Choice',valid_Editor_Choice, df_2)]

valid_In_app_purchases= [True, False]
df_2=df2[df2['In app purchases' ].notna()]
invalids["In app purchases"] = df_2[~validate_isin('In app purchases',valid_In_app_purchases, df_2)]

df_2=df2[df2['Reviews' ].notna()]
invalids['Reviews'] = df_2[~validate_bounded('Reviews',0,np.inf, df_2)]

valid_Ad_Supported= [True]
df_2=df2[df2['Ad Supported' ].notna()]
invalids["Ad Supported"] = df_2[~validate_isin('Ad Supported',valid_Ad_Supported, df_2)]

valid_Content_Rating= ['Everyone','Teen', 'Mature 17+', 'Everyone 10+', 'Adults only 18+', 'Unrated']
df_2=df2[df2['Content Rating' ].notna()]
invalids['Content Rating'] = df_2[~validate_isin('Content Rating',valid_Content_Rating, df_2)]

df_2=df2[df2['Last update' ].notna()]
valid_Last_update	= '\d{1,2}-[(Jan)|(Aug)|Dec|Feb|Jul|Apr|Aug|Mar|May|Nov|Sep|Oct]{3}-\d{2}'
invalids['Last update'] = df_2[~validate_pattern('Last update',valid_Last_update	, df_2)]

df_2=df2[df2['Released' ].notna()]
valid_Released= '\d{1,2}-[(Jan)|(Aug)|Dec|Feb|Jul|Apr|Aug|Mar|May|Nov|Sep|Oct]{3}-\d{2}'
invalids['Released'] = df_2[~validate_pattern('Released',valid_Released, df_2)]

df_2=df2[df2['Developer Email' ].notna()]
valid_Developer_Email= '[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}(?:\.[A-Za-z]{2,})?'
invalids['Developer Email'] = df_2[~validate_pattern('Developer Email',valid_Developer_Email, df_2)]

df_2=df2[df2['Privacy Policy' ].notna()]
valid_Privacy_Policy= '[hH][tT][tT][pP][sS]?://.+'
invalids['Privacy Policy'] = df_2[~validate_pattern('Privacy Policy',valid_Privacy_Policy, df_2)]

df_2=df2[df2['Developer Website' ].notna()]
valid_Developer_Website= '[hH][tT][tT][pP][sS]?://.+'
invalids['Developer Website'] = df_2[~validate_pattern('Developer Website',valid_Developer_Website, df_2)]

df_2=df2[df2['Minimum Android' ].notna()]
valid_Minimum_Android= '\d+(\.\d+)*'
invalids['Minimum Android'] = df_2[~validate_pattern('Minimum Android',valid_Minimum_Android, df_2)]

df_2=df2[df2['Installs' ].notna()]
valid_Installs= '(?:\b\d{1,3}(?:,\d{3})*\+\b)'
invalids['Installs'] = df_2[~validate_pattern('Installs',valid_Installs, df_2)]

df_2=df2[df2['Size' ].notna()]
valid_Size= '\d{1,3}(?:,\d{3})*(?:\.\d+)?[kM]'
invalids['Size'] = df_2[~validate_pattern('Size',valid_Size, df_2)]

df_2=df2[df2['Currency' ].notna()]
valid_Currency= '[A-Z]{3}'
invalids['Currency'] = df_2[~validate_pattern('Currency',valid_Currency, df_2)]

df_2=df2[df2['Price' ].notna()]
df_2['Price'] = df_2['Price'].astype(str)
valid_Price= '(?:\d+(?:\.\d+)?)'
invalids['Price'] = df_2[~validate_pattern('Price',valid_Price, df_2)]

df_2=df2[df2['Minimum Installs' ].notna()]
df_2['Minimum Installs'] = df_2['Minimum Installs'].astype(str)
valid_Minimum_Installs= '(?:\d+(?:\.\d+)?)'
invalids['Minimum Installs'] = df_2[~validate_pattern('Minimum Installs',valid_Minimum_Installs, df_2)]


df_2=df2[df2['Rating' ].notna()]
invalids['Rating'] = df_2[~validate_bounded('Rating',0,5, df_2)]

df_2=df2[df2['Category' ].notna()]
valid_Category = '(?:Education|Tools|Books & Reference|Personalization|Music & Audio|Entertainment|Lifestyle|Business|Productivity|Health & Fitness|Puzzle|Travel & Local|Casual|Photography|Sports|Finance|Simulation|News & Magazines|Arcade|Shopping|Food & Drink|Communication|Action|Educational|Social|Medical|Adventure|Maps & Navigation|Role Playing|Video Players & Editors|Art & Design|Racing|Board|Strategy|Card|Word|Auto & Vehicles|Weather|Trivia|House & Home|Beauty|Dating|Casino|Libraries & Demo|Events|Music|Parenting|Comics)'
invalids['Category'] = df_2[~validate_pattern('Category',valid_Category, df_2)]

df_2=df2[df2['Rating Count' ].notna()]
df_2['Rating Count'] = df_2['Rating Count'].astype(str)
valid_Rating_Count= '\d+\.\d+'
invalids['Rating Count'] = df_2[~validate_pattern('Rating Count',valid_Rating_Count, df_2)]


C:\Users\BlueWindows\AppData\Local\Temp\ipykernel_3196\2401283195.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['Developer Internal ID'] = df_2['Developer Internal ID'].astype(str)
C:\Users\BlueWindows\AppData\Local\Temp\ipykernel_3196\2401283195.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['Price'] = df_2['Price'].astype(str)
C:\Users\BlueWindows\AppData\Local\Temp\ipykernel_3196\2401283195.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [19]:
df2 = df2.drop(columns=df2.filter(like='Unnamed').columns)
sum2=0
quality_df2 = pd.DataFrame(index=list(df2.columns),columns=['Consistency', 'Currentness', 'Validity', 'Completeness', 'Accuracy', 'Null Values', 'Records'])

for column in df2.columns:
    quality_df2['Records'][column] = len(df2)

for column in df2.columns:
    if 'Unnamed' not in column:
        quality_df2['Null Values'][column] = df2[column].isnull().sum()
        missing = df2[column].isnull().sum()
        if column != 'App Name' and column != 'App Id' and column != 'Developer Id' and column != 'Summary' and column != 'Developer' and column != 'Developer Address' and column != 'Version':
            sum_acc = ((len(df2) - (len(invalids[column]) + missing)) / len(df2)) * 100
            quality_df2['Accuracy'][column] = sum_acc
            sum = (((len(df2[column]) - missing) / len(df2[column])) * 100)
            quality_df2['Completeness'][column] = sum
            sum =( ((len(df2) - (len(invalids[column])+ missing) )/ (len(df2)-missing)) * 100)
            quality_df2['Validity'][column]=sum

valid_dates2 = df2[validate_pattern('Last update',valid_Last_update, df2)]
valid_dates2['Year'] = valid_dates2['Last update'].apply(lambda x: int(x[-2:]))
valid_dates2_before_2018 = valid_dates2[valid_dates2['Year'] < 17]
percentage_valid2_before_2017 = (len(valid_dates_before_2018) / len(valid_dates)) * 100
for column in df2.columns:
    if column != 'Unnamed: 0':  
        quality_df2['Currentness'][column]=percentage_valid2_before_2017

quality_df2.to_csv('output2.csv')

quality_df2

C:\Users\BlueWindows\AppData\Local\Temp\ipykernel_3196\3502426032.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_dates2['Year'] = valid_dates2['Last update'].apply(lambda x: int(x[-2:]))


,Consistency,Currentness,Validity,Completeness,Accuracy,Null Values,Records
App Name,NaN,14.98155,NaN,NaN,NaN,13,450794
App Id,NaN,14.98155,NaN,NaN,NaN,0,450794
Category,NaN,14.98155,100.0,99.997116,99.997116,13,450794
Rating,NaN,14.98155,98.147247,99.376212,97.535016,2812,450794
Rating Count,NaN,14.98155,100.0,73.653598,73.653598,118768,450794
Installs,NaN,14.98155,0.0,99.979813,0.0,91,450794
Minimum Installs,NaN,14.98155,100.0,99.997116,99.997116,13,450794
Free,NaN,14.98155,100.0,99.979592,99.979592,92,450794
Price,NaN,14.98155,100.0,99.979592,99.979592,92,450794
Currency,NaN,14.98155,100.0,99.979592,99.979592,92,450794


comsistancy functions

In [20]:
invalid_prices = df2[(df2['Free'] == 'TRUE') & (df2['Price'] != 0)]
print(invalid_prices)


Empty DataFrame
Columns: [App Name, App Id, Category, Rating, Rating Count, Installs, Minimum Installs, Free, Price, Currency, Size, Minimum Android, Developer Id, Developer Website, Developer Email, Released, Last update, Privacy Policy, Content Rating, Ad Supported, In app purchases, Editor Choice, Summary, Reviews, Android version Text, Developer, Developer Address, Developer Internal ID, Version]
Index: []

[0 rows x 29 columns]


In [21]:

df2['Installs'] = df2['Installs'].fillna('0')

df2['Installs_numeric'] = df2['Installs'].str.replace('[+,]', '', regex=True).astype('int64')

inconsistent_installs = df2[df2['Installs_numeric'] != df2['Minimum Installs']]

print("Rows with inconsistent 'Installs' and 'Minimum Installs':")
print(inconsistent_installs)


Rows with inconsistent 'Installs' and 'Minimum Installs':
       App Name                                             App Id Category   
13103       NaN                   id.akusantri.wallpapermuslimahhd      NaN  \
110672      NaN                         pl.rosmedia.rozmowkirosang      NaN   
110733      NaN                         com.web.translatemasterpro      NaN   
110846      NaN                           com.eurotalk.utalk.latin      NaN   
110862      NaN                        com.eurotalk.utalk.galician      NaN   
110898      NaN                             com.kirson.fruitbubble      NaN   
110910      NaN                            com.bigknol.hindidoodle      NaN   
111035      NaN  com.gapps.learning.center.cyber.security.learning      NaN   
245437      NaN                              pl.alior.traderMobile      NaN   
291626      NaN                                com.aramex.ecourier      NaN   
291897      NaN                             com.winrgames.letsleap      N

In [22]:

df2 = df2.drop(columns=df2.filter(like='Unnamed').columns)

df2['Released'] = pd.to_datetime(df2['Released'], errors='coerce')
df2['Last update'] = pd.to_datetime(df2['Last update'], errors='coerce')

parsing_errors = df2[df2['Released'].isnull() | df2['Last update'].isnull()]
print("Rows with parsing errors:")
print(parsing_errors)
print('*'*50)

df2.dropna(subset=['Released', 'Last update'], inplace=True)

invalid_rows = df2[df2['Last update'] < df2['Released']]

invalid_rows = invalid_rows.dropna(subset=['Released', 'Last update'])

print("Rows where Last update is before Released after removing NaN or null:")
print(invalid_rows)


C:\Users\BlueWindows\AppData\Local\Temp\ipykernel_3196\4016345658.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['Released'] = pd.to_datetime(df2['Released'], errors='coerce')
C:\Users\BlueWindows\AppData\Local\Temp\ipykernel_3196\4016345658.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['Last update'] = pd.to_datetime(df2['Last update'], errors='coerce')


Rows with parsing errors:
                                                 App Name   
197                                   Mazes: Puzzle Games  \
321                            Kids Numbers and Math FREE   
1208    Nithra STEPS |Connecting Students,Teachers&Par...   
1276                                Cytaty na każdy dzień   
1277                          Tasbeeh Counter Zikr Tasbih   
...                                                   ...   
450236                             German Car Plates Free   
450237                                   Web Monitor Free   
450240                                        SpeedoMeter   
450779                                        Demo Parent   
450792                 Meetz - Cloud Meetings and Classes   

                                                   App Id           Category   
197     com.rvappstudios.maze.games.puzzle.mazes.labyr...             Puzzle  \
321                                   zok.android.numbers        Educational   
1

In [23]:

df2['Installs'] = df2['Installs'].fillna('0')
df2['Installs_numeric'] = df2['Installs'].str.replace('[+,]', '', regex=True).astype('int64')

for column in df2.columns:
    if 'Unnamed' not in column:
        if column == 'Installs':
            consistent_installs = df2[df2['Installs_numeric'] >= df2['Minimum Installs']]
            consistency_percentage = (len(consistent_installs) / len(df2)) * 100
            quality_df2['Consistency']['Installs'] = consistency_percentage

consistent_dates = df2[df2['Released'] <= df2['Last update']]
consistency_percentage_dates = (len(consistent_dates) / len(df2)) * 100
quality_df2['Consistency']['Released'] = consistency_percentage_dates
quality_df2['Consistency']['Last update'] = consistency_percentage_dates

consistent_prices = df2[(df2['Free'] == 'TRUE') | (df2['Price'] == 0)]
consistency_percentage_prices = (len(consistent_prices) / len(df2)) * 100
quality_df2['Consistency']['Price'] = consistency_percentage_prices
quality_df2['Consistency']['Free'] = consistency_percentage_prices
quality_df2 = quality_df2.fillna(100)

print(quality_df2)


                       Consistency  Currentness    Validity  Completeness   
App Name                100.000000     14.98155  100.000000    100.000000  \
App Id                  100.000000     14.98155  100.000000    100.000000   
Category                100.000000     14.98155  100.000000     99.997116   
Rating                  100.000000     14.98155   98.147247     99.376212   
Rating Count            100.000000     14.98155  100.000000     73.653598   
Installs                100.000000     14.98155    0.000000     99.979813   
Minimum Installs        100.000000     14.98155  100.000000     99.997116   
Free                     93.799571     14.98155  100.000000     99.979592   
Price                    93.799571     14.98155  100.000000     99.979592   
Currency                100.000000     14.98155  100.000000     99.979592   
Size                    100.000000     14.98155   95.195733     99.993789   
Minimum Android         100.000000     14.98155   97.244681     99.759979   